In [ ]:
#########################################
## granite-3.3-2b-instruct en Colab
#########################################

In [ ]:
# installations

import requests, os, re, torch, ast, time

from transformers import AutoTokenizer, AutoModelForCausalLM

from transformers.utils import get_json_schema

from datetime import datetime, timezone, timedelta


In [ ]:
TOKENIZER = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")

GOOGLE_API_KEY = os.getenv('GOOGLE API KEY',"xxxxxxxxxxxxxxx")

model_path="ibm-granite/granite-3.3-2b-instruct"

device="cuda"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

In [ ]:
def get_geolocation() :
  return "Avenida Complutense 30, Madrid, España"

def geocode(address):
    if address == 'None':
      address = get_geolocation()
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GOOGLE_API_KEY}"
    response = requests.get(geocode_url)
    results = response.json()["results"]
    if not results:
        raise ValueError(f"No se encontró la dirección: {address}")
    location = results[0]["geometry"]["location"]
    return location["lat"], location["lng"]


def get_route_duration(origin: 'None', end: 'None') -> dict:

    """
    Devuelve la duración del trayecto en automóvil desde una dirección de origen a una de destino, y un valor que indica si hay atasco
    en el trayecto.

    Args:

    origin: el punto de comienzo, en formato cadena de caracteres, por ejemplo, "calle de Torres Quevedo 189, Madrid, España"

    end: destino del trayecto, en formato cadena de caracteres.

    Returns:

    Un diccionario con la respuesta de duración del trayecto y un indicador sobre si hay atasco en la ruta.

    """

    print(f"Calculando distancia y duración desde {origin} a {end}")

    try:

      lat_origen, lon_origen = geocode(origin)
      lat_destino, lon_destino = geocode(end)
      departure_dt = datetime.now(timezone.utc) + timedelta(minutes=1)
      departure_iso_string = departure_dt.strftime('%Y-%m-%dT%H:%M:%SZ')

      route_url = "https://routes.googleapis.com/directions/v2:computeRoutes"
      headers = {
       "Content-Type": "application/json",
       "X-Goog-Api-Key": GOOGLE_API_KEY,
        "X-Goog-FieldMask": "*"
      }
      # Primero calculamos la duración del trayecto sin tráfico
      route_data = {
        "origin": {
            "location": {
                "latLng": {"latitude": lat_origen, "longitude": lon_origen}
            }
        },
        "destination": {
          "location": {
              "latLng": {"latitude": lat_destino, "longitude": lon_destino}
          }
        },
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_UNAWARE",
        "computeAlternativeRoutes": False,
        "routeModifiers": {
            "avoidTolls": True,
            "avoidHighways": False,
            "avoidFerries": True
        },
        "languageCode": "es-ES",
        "units": "METRIC"
      }
      route_response = requests.post(route_url, headers=headers, json=route_data)
      route_duration_non_traffic = int(route_response.json()["routes"][0]["duration"].rstrip("s"))

      # Calculamos el tiempo del trayecto con tráfico

      route_data = {
        "origin": {
            "location": {
                "latLng": {"latitude": lat_origen, "longitude": lon_origen}
            }
        },
        "destination": {
          "location": {
              "latLng": {"latitude": lat_destino, "longitude": lon_destino}
          }
        },
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE",
        "departureTime": departure_iso_string,
        "computeAlternativeRoutes": False,
        "routeModifiers": {
            "avoidTolls": True,
            "avoidHighways": False,
            "avoidFerries": True
        },
        "languageCode": "es-ES",
        "units": "METRIC"
      }
      route_response = requests.post(route_url, headers=headers, json=route_data)
      route_duration_traffic = int(route_response.json()["routes"][0]["duration"].rstrip("s"))

      print("segundos tráfico", route_duration_traffic)
      print("segundos non tráfico", route_duration_non_traffic)
      if route_duration_traffic >= route_duration_non_traffic*1.1 :
        traffic_jam = 'yes' # añadimos un item más al diccionario
      else :
        traffic_jam = 'no'

      return {

            "duration": round(route_duration_traffic/60), # damos la duración en minutos

            "traffic_jam": traffic_jam
      }

    except Exception as e:

        print(f"Error al buscar información en Google Maps: {e}")

        return {

            "duration": "none",

            "traffic_jam": "none"
        }


In [ ]:
from transformers.utils import get_json_schema
tools = [get_json_schema(get_route_duration)]
tools

[{'type': 'function',
  'function': {'name': 'get_route_duration',
   'description': 'Devuelve la duración del trayecto en automóvil desde una dirección de origen a una de destino, y un valor que indica si hay atasco\nen el trayecto.',
   'parameters': {'type': 'object',
    'properties': {'origin': {'type': 'null',
      'description': 'el punto de comienzo, en formato cadena de caracteres, por ejemplo, "calle de Torres Quevedo 189, Madrid, España"'},
     'end': {'type': 'null',
      'description': 'destino del trayecto, en formato cadena de caracteres.'}},
    'required': ['origin', 'end']},
   'return': {'type': 'object',
    'description': 'Un diccionario con la respuesta de duración del trayecto y un indicador sobre si hay atasco en la ruta.'}}}]

In [ ]:
def make_api_request(instructions) -> str:

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map=device,
        torch_dtype=torch.bfloat16,
    )

    model_parameters = {

        "do_sample": False, # es equivalente a "decoding_method": "greedy". Me da como salida el token más probable.

        "max_new_tokens": 2000,

        "repetition_penalty": 1.05,

        "eos_token_id" : TOKENIZER.eos_token_id # es equivalente a "stop_sequences": [TOKENIZER.eos_token] y sirve para indicarle al modelo
                                                # cuándo parar
    }

    output = model.generate(
        **instructions,
        **model_parameters,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_path)

    prediction = tokenizer.decode(output[0], skip_special_tokens=False)
    print("prediction", prediction)
    print("***********************************************************************")
    prediction = tokenizer.decode(output[0, instructions["input_ids"].shape[1]:], skip_special_tokens=True)

    return prediction

In [ ]:
question= "¿Cómo está el tráfico para llegar a la Plaza de España en Madrid?"
#question= "¿Hay atasco para ir a la Plaza de España en Madrid?"
#question= "¿Cómo de densa está la circulación para ir a la Plaza de España de Madrid?"
query = question+" Teniendo en cuenta que el origen es "+ get_geolocation()

conversation = [{"role": "system","content": """Eres un asistente con acceso a las siguientes llamadas a función.
Tu tarea es producir una secuencia de llamadas a función en respuesta a la petición del usuario.
Muestra la respuesta en lenguaje natural, en español.
En tu resultado escribe los argumentos de la función tras "arguments" y el nombre de la función tras "function"
"""},

{"role": "user", "content": query},
]

instructions = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, tokenize=False, add_generation_prompt=False)
instructions

'<|start_of_role|>system<|end_of_role|>Eres un asistente con acceso a las siguientes llamadas a función.\nTu tarea es producir una secuencia de llamadas a función en respuesta a la petición del usuario.\nMuestra la respuesta en lenguaje natural, en español.\nEn tu resultado escribe los argumentos de la función tras "arguments" y el nombre de la función tras "function"\n<|end_of_text|>\n<|start_of_role|>available_tools<|end_of_role|>[\n    {\n        "type": "function",\n        "function": {\n            "name": "get_route_duration",\n            "description": "Devuelve la duración del trayecto en automóvil desde una dirección de origen a una de destino, y un valor que indica si hay atasco\\nen el trayecto.",\n            "parameters": {\n                "type": "object",\n                "properties": {\n                    "origin": {\n                        "type": "null",\n                        "description": "el punto de comienzo, en formato cadena de caracteres, por ejemplo, 

In [ ]:
instructions = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, return_tensors="pt", return_dict=True, add_generation_prompt=True).to(device)
instructions

{'input_ids': tensor([[49152,  2946, 49153,    55,   575,   707,   619,  5443,    87,   457,
         41904,   312,  9102, 42666, 48529, 11660,   312, 36426,    32,   203,
         31748,   273,  3768,  2813,   534,  3881,   476,  7542,  8382,   103,
          9820,   409, 48529, 11660,   312, 36426,   931,  5127, 26919,   312,
          2317,   298, 19399,  4492,  3555, 17796,    32,   203,    63,   340,
         11560,  2317,  5127, 26919,   931,   456,   819,   103, 13880, 19844,
            30,   931,  2813,  4535, 10657,   362,    32,   203,   776, 21928,
         33663,  2813,  6072,  6957,  2793,   500,   409,  2317, 36426,   554,
           302,   313,  8299,    20,   533,   709, 16865,   409,  2317, 36426,
           554,   302,   313,  1126,    20,   203,     0,   203, 49152,  7230,
            81,  4280, 49153,    77,   284,   301,   291,   313,   681,   563,
           313,  1126,   392,   291,   313,  1126,   563,   301,   324,   313,
           426,   563,   313,   371,  

In [ ]:
data_traffic = make_api_request(instructions)

data_traffic



config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

prediction <|start_of_role|>system<|end_of_role|>Eres un asistente con acceso a las siguientes llamadas a función.
Tu tarea es producir una secuencia de llamadas a función en respuesta a la petición del usuario.
Muestra la respuesta en lenguaje natural, en español.
En tu resultado escribe los argumentos de la función tras "arguments" y el nombre de la función tras "function"
<|end_of_text|>
<|start_of_role|>available_tools<|end_of_role|>[
    {
        "type": "function",
        "function": {
            "name": "get_route_duration",
            "description": "Devuelve la duración del trayecto en automóvil desde una dirección de origen a una de destino, y un valor que indica si hay atasco\nen el trayecto.",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {
                        "type": "null",
                        "description": "el punto de comienzo, en formato cadena de caracteres, por ejemplo, \"calle

'[{"arguments": {"origin": "Avenida Complutense 30, Madrid, España", "end": "Plaza de España, Madrid, España"}, "name": "get_route_duration"}]'

In [ ]:
def tool_call(llm_response: str):

    tool_request = ast.literal_eval(re.search("({.+})", llm_response, re.DOTALL).group(0)) #re.DOTALL detecta todos los caracteres incluidos los saltos de linea
    print(tool_request)
    tool_name = tool_request["name"]

    tool_arguments = tool_request["arguments"]

    tool_response = globals()[tool_name](**tool_arguments)

    return tool_response


In [ ]:
tool_response_traffic = tool_call(data_traffic)

tool_response_traffic


{'arguments': {'origin': 'Avenida Complutense 30, Madrid, España', 'end': 'Plaza de España, Madrid, España'}, 'name': 'get_route_duration'}
Calculando distancia y duración desde Avenida Complutense 30, Madrid, España a Plaza de España, Madrid, España
segundos tráfico 764
segundos non tráfico 740


{'duration': 13, 'traffic_jam': 'no'}

In [ ]:
conversation2 =  [

    {"role": "system", "content":
    """
    Muestra la respuesta de la función externa en lenguaje natural, en español.
    """},

    {"role": "user", "content":
    """Haz que la respuesta sea muy breve.
    Tu respuesta debe ser muy corta, debes decir: "No hay atasco" si traffic_jam es 'no'. E incluir también en la respuesta la duración de trayecto
     en minutos.
    Tu respuesta debe ser muy corta, debes decir: "Hay atasco" si traffic_jam es 'yes'. E incluir también en la respuesta la duración de trayecto en minutos.
    No incluyas nombres de calles en tu respuesta
    No incluyas nombres de ciudades o pueblos en tu respuesta
    No incluyas nombres de países en tu respuesta
    """ },

    {"role": "tool_response", "content": str(tool_response_traffic) },

]

instruction_2 = TOKENIZER.apply_chat_template(conversation=conversation2, tokenize=False, add_generation_prompt=False)
instruction_2


'<|start_of_role|>system<|end_of_role|>\n    Muestra la respuesta de la función externa en lenguaje natural, en español.\n    <|end_of_text|>\n<|start_of_role|>user<|end_of_role|>Haz que la respuesta sea muy breve.\n    Tu respuesta debe ser muy corta, debes decir: "No hay atasco" si traffic_jam es \'no\'. E incluir también en la respuesta la duración de trayecto\n     en minutos.\n    Tu respuesta debe ser muy corta, debes decir: "Hay atasco" si traffic_jam es \'yes\'. E incluir también en la respuesta la duración de trayecto en minutos.\n    No incluyas nombres de calles en tu respuesta\n    No incluyas nombres de ciudades o pueblos en tu respuesta\n    No incluyas nombres de países en tu respuesta\n    <|end_of_text|>\n<|start_of_role|>tool_response<|end_of_role|>{\'duration\': 13, \'traffic_jam\': \'no\'}<|end_of_text|>\n'

In [ ]:
instructions_2 = TOKENIZER.apply_chat_template(conversation=conversation2, return_tensors="pt", return_dict=True, add_generation_prompt=True).to(device)

data_2 = make_api_request(instructions_2)

data_2


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

prediction <|start_of_role|>system<|end_of_role|>
    Muestra la respuesta de la función externa en lenguaje natural, en español.
    <|end_of_text|>
<|start_of_role|>user<|end_of_role|>Haz que la respuesta sea muy breve.
    Tu respuesta debe ser muy corta, debes decir: "No hay atasco" si traffic_jam es 'no'. E incluir también en la respuesta la duración de trayecto
     en minutos.
    Tu respuesta debe ser muy corta, debes decir: "Hay atasco" si traffic_jam es 'yes'. E incluir también en la respuesta la duración de trayecto en minutos.
    No incluyas nombres de calles en tu respuesta
    No incluyas nombres de ciudades o pueblos en tu respuesta
    No incluyas nombres de países en tu respuesta
    <|end_of_text|>
<|start_of_role|>tool_response<|end_of_role|>{'duration': 13, 'traffic_jam': 'no'}<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>No hay atasco. Duración del trayecto: 13 minutos.<|end_of_text|>
*********************************************************************

'No hay atasco. Duración del trayecto: 13 minutos.'

In [ ]:
conversation3 =  [

    {"role": "system", "content":
    """
    Muestra la respuesta de la función externa en lenguaje natural, en español.
    """},

    {"role": "user", "content":
    """Haz que la respuesta sea muy breve.
    Tu respuesta debe ser muy corta, debes decir: "No hay atasco" si traffic_jam es 'no'.
    Tu respuesta debe ser muy corta, debes decir: "Hay atasco" si traffic_jam es 'yes'.
    Responde en 3 palabras
    La respuesta debería ser: "No hay atasco" o "Hay atasco"
    No incluyas nombres de calles en tu respuesta
    No incluyas nombres de ciudades o pueblos en tu respuesta
    No incluyas nombres de países en tu respuesta
    """ },

    {"role": "tool_response", "content": str(tool_response_traffic) },

]

instruction_3 = TOKENIZER.apply_chat_template(conversation=conversation3, tokenize=False, add_generation_prompt=False)
instruction_3


'<|start_of_role|>system<|end_of_role|>\n    Muestra la respuesta de la función externa en lenguaje natural, en español.\n    <|end_of_text|>\n<|start_of_role|>user<|end_of_role|>Haz que la respuesta sea muy breve.\n    Tu respuesta debe ser muy corta, debes decir: "No hay atasco" si traffic_jam es \'no\'.\n    Tu respuesta debe ser muy corta, debes decir: "Hay atasco" si traffic_jam es \'yes\'.\n    Responde en 3 palabras\n    La respuesta debería ser: "No hay atasco" o "Hay atasco"\n    No incluyas nombres de calles en tu respuesta\n    No incluyas nombres de ciudades o pueblos en tu respuesta\n    No incluyas nombres de países en tu respuesta\n    <|end_of_text|>\n<|start_of_role|>tool_response<|end_of_role|>{\'duration\': 13, \'traffic_jam\': \'no\'}<|end_of_text|>\n'

In [ ]:
instructions_3 = TOKENIZER.apply_chat_template(conversation=conversation3, return_tensors="pt", return_dict=True, add_generation_prompt=True).to(device)
data_3 = make_api_request(instructions_3)

data_3

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

prediction <|start_of_role|>system<|end_of_role|>
    Muestra la respuesta de la función externa en lenguaje natural, en español.
    <|end_of_text|>
<|start_of_role|>user<|end_of_role|>Haz que la respuesta sea muy breve.
    Tu respuesta debe ser muy corta, debes decir: "No hay atasco" si traffic_jam es 'no'.
    Tu respuesta debe ser muy corta, debes decir: "Hay atasco" si traffic_jam es 'yes'.
    Responde en 3 palabras
    La respuesta debería ser: "No hay atasco" o "Hay atasco"
    No incluyas nombres de calles en tu respuesta
    No incluyas nombres de ciudades o pueblos en tu respuesta
    No incluyas nombres de países en tu respuesta
    <|end_of_text|>
<|start_of_role|>tool_response<|end_of_role|>{'duration': 13, 'traffic_jam': 'no'}<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>No hay atasco.<|end_of_text|>
***********************************************************************


'No hay atasco.'